# Example Notebook

Welcome to the example notebook for the Home Credit Kaggle competition. The goal of this competition is to determine how likely a customer is going to default on an issued loan. The main difference between the [first](https://www.kaggle.com/c/home-credit-default-risk) and this competition is that now your submission will be scored with a custom metric that will take into account how well the model performs in future. A decline in performance will be penalized. The goal is to create a model that is stable and performs well in the future.

In this notebook you will see how to:
* Load the data
* Join tables with Polars - a DataFrame library implemented in Rust language, designed to be blazingy fast and memory efficient.  
* Create simple aggregation features
* Train a LightGBM model
* Create a submission table

## Load the data

In [2]:
import polars as pl
import numpy as np
import pandas as pd

# import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

dataPath = "/kaggle/input/home-credit-credit-risk-model-stability/"

In [3]:
def set_table_dtypes(df: pl.DataFrame) -> pl.DataFrame:
    # implement here all desired dtypes for tables
    # the following is just an example
    for col in df.columns:
        # last letter of column name will help you determine the type
        if col[-1] in ("P", "A"):
            df = df.with_columns(pl.col(col).cast(pl.Float64).alias(col))

    return df


def convert_strings(df: pd.DataFrame) -> pd.DataFrame:
    for col in df.columns:
        if df[col].dtype.name in ["object", "string"]:
            df[col] = df[col].astype("string").astype("category")
            current_categories = df[col].cat.categories
            new_categories = current_categories.to_list() + ["Unknown"]
            new_dtype = pd.CategoricalDtype(categories=new_categories, ordered=True)
            df[col] = df[col].astype(new_dtype)
    return df

In [4]:
train_basetable = pl.read_csv("train_base.csv")
train_static = pl.concat(
    [
        pl.read_csv("train_static_0_0.csv").pipe(set_table_dtypes),
        pl.read_csv("train_static_0_1.csv").pipe(set_table_dtypes),
    ],
    how="vertical_relaxed",
)
train_static_cb = pl.read_csv("train_static_cb_0.csv").pipe(set_table_dtypes)
train_person_1 = pl.read_csv("train_person_1.csv").pipe(set_table_dtypes)
train_credit_bureau_b_2 = pl.read_csv("train_credit_bureau_b_2.csv").pipe(
    set_table_dtypes
)

In [5]:
test_basetable = pl.read_csv("test_base.csv")
test_static = pl.concat(
    [
        pl.read_csv("test_static_0_0.csv").pipe(set_table_dtypes),
        pl.read_csv("test_static_0_1.csv").pipe(set_table_dtypes),
        pl.read_csv("test_static_0_2.csv").pipe(set_table_dtypes),
    ],
    how="vertical_relaxed",
)
test_static_cb = pl.read_csv("test_static_cb_0.csv").pipe(set_table_dtypes)
test_person_1 = pl.read_csv("test_person_1.csv").pipe(set_table_dtypes)
test_credit_bureau_b_2 = pl.read_csv("test_credit_bureau_b_2.csv").pipe(
    set_table_dtypes
)

## Feature engineering

In this part, we can see a simple example of joining tables via `case_id`. Here the loading and joining is done with polars library. Polars library is blazingly fast and has much smaller memory footprint than pandas. 

In [6]:
# We need to use aggregation functions in tables with depth > 1, so tables that contain num_group1 column or
# also num_group2 column.
train_person_1_feats_1 = train_person_1.group_by("case_id").agg(
    pl.col("mainoccupationinc_384A").max().alias("mainoccupationinc_384A_max"),
    (pl.col("incometype_1044T") == "SELFEMPLOYED")
    .max()
    .alias("mainoccupationinc_384A_any_selfemployed"),
)

# Here num_group1=0 has special meaning, it is the person who applied for the loan.
train_person_1_feats_2 = (
    train_person_1.select(["case_id", "num_group1", "housetype_905L"])
    .filter(pl.col("num_group1") == 0)
    .drop("num_group1")
    .rename({"housetype_905L": "person_housetype"})
)

# Here we have num_goup1 and num_group2, so we need to aggregate again.
train_credit_bureau_b_2_feats = train_credit_bureau_b_2.group_by("case_id").agg(
    pl.col("pmts_pmtsoverdue_635A").max().alias("pmts_pmtsoverdue_635A_max"),
    (pl.col("pmts_dpdvalue_108P") > 31).max().alias("pmts_dpdvalue_108P_over31"),
)

# We will process in this examples only A-type and M-type columns, so we need to select them.
selected_static_cols = []
for col in train_static.columns:
    if col[-1] in ("A", "M"):
        selected_static_cols.append(col)
print(selected_static_cols)

selected_static_cb_cols = []
for col in train_static_cb.columns:
    if col[-1] in ("A", "M"):
        selected_static_cb_cols.append(col)
print(selected_static_cb_cols)

# Join all tables together.
data = (
    train_basetable.join(
        train_static.select(["case_id"] + selected_static_cols),
        how="left",
        on="case_id",
    )
    .join(
        train_static_cb.select(["case_id"] + selected_static_cb_cols),
        how="left",
        on="case_id",
    )
    .join(train_person_1_feats_1, how="left", on="case_id")
    .join(train_person_1_feats_2, how="left", on="case_id")
    .join(train_credit_bureau_b_2_feats, how="left", on="case_id")
)

['amtinstpaidbefduel24m_4187115A', 'annuity_780A', 'annuitynextmonth_57A', 'avginstallast24m_3658937A', 'avglnamtstart24m_4525187A', 'avgoutstandbalancel6m_4187114A', 'avgpmtlast12m_4525200A', 'credamount_770A', 'currdebt_22A', 'currdebtcredtyperange_828A', 'disbursedcredamount_1113A', 'downpmt_116A', 'inittransactionamount_650A', 'lastapprcommoditycat_1041M', 'lastapprcommoditytypec_5251766M', 'lastapprcredamount_781A', 'lastcancelreason_561M', 'lastotherinc_902A', 'lastotherlnsexpense_631A', 'lastrejectcommoditycat_161M', 'lastrejectcommodtypec_5251769M', 'lastrejectcredamount_222A', 'lastrejectreason_759M', 'lastrejectreasonclient_4145040M', 'maininc_215A', 'maxannuity_159A', 'maxannuity_4075009A', 'maxdebt4_972A', 'maxinstallast24m_3658928A', 'maxlnamtstart6m_4525199A', 'maxoutstandbalancel12m_4187113A', 'maxpmtlast3m_4525190A', 'previouscontdistrict_112M', 'price_1097A', 'sumoutstandtotal_3546847A', 'sumoutstandtotalest_4493215A', 'totaldebt_9A', 'totalsettled_863A', 'totinstallas

In [7]:
# subset for where week is 0

data_0 = data.filter(pl.col("case_id") == 405)

In [8]:
test_person_1_feats_1 = test_person_1.group_by("case_id").agg(
    pl.col("mainoccupationinc_384A").max().alias("mainoccupationinc_384A_max"),
    (pl.col("incometype_1044T") == "SELFEMPLOYED")
    .max()
    .alias("mainoccupationinc_384A_any_selfemployed"),
)

test_person_1_feats_2 = (
    test_person_1.select(["case_id", "num_group1", "housetype_905L"])
    .filter(pl.col("num_group1") == 0)
    .drop("num_group1")
    .rename({"housetype_905L": "person_housetype"})
)

test_credit_bureau_b_2_feats = test_credit_bureau_b_2.group_by("case_id").agg(
    pl.col("pmts_pmtsoverdue_635A").max().alias("pmts_pmtsoverdue_635A_max"),
    (pl.col("pmts_dpdvalue_108P") > 31).max().alias("pmts_dpdvalue_108P_over31"),
)

data_submission = (
    test_basetable.join(
        test_static.select(["case_id"] + selected_static_cols), how="left", on="case_id"
    )
    .join(
        test_static_cb.select(["case_id"] + selected_static_cb_cols),
        how="left",
        on="case_id",
    )
    .join(test_person_1_feats_1, how="left", on="case_id")
    .join(test_person_1_feats_2, how="left", on="case_id")
    .join(test_credit_bureau_b_2_feats, how="left", on="case_id")
)

In [9]:
case_ids = data["case_id"].unique().shuffle(seed=1)
case_ids_train, case_ids_test = train_test_split(
    case_ids, train_size=0.6, random_state=1
)
case_ids_valid, case_ids_test = train_test_split(
    case_ids_test, train_size=0.5, random_state=1
)

cols_pred = []
for col in data.columns:
    if col[-1].isupper() and col[:-1].islower():
        cols_pred.append(col)

print(cols_pred)


def from_polars_to_pandas(case_ids: pl.DataFrame) -> pl.DataFrame:
    return (
        data.filter(pl.col("case_id").is_in(case_ids))[
            ["case_id", "WEEK_NUM", "target"]
        ].to_pandas(),
        data.filter(pl.col("case_id").is_in(case_ids))[cols_pred].to_pandas(),
        data.filter(pl.col("case_id").is_in(case_ids))["target"].to_pandas(),
    )


base_train, X_train, y_train = from_polars_to_pandas(case_ids_train)
base_valid, X_valid, y_valid = from_polars_to_pandas(case_ids_valid)
base_test, X_test, y_test = from_polars_to_pandas(case_ids_test)

for df in [X_train, X_valid, X_test]:
    df = convert_strings(df)

['amtinstpaidbefduel24m_4187115A', 'annuity_780A', 'annuitynextmonth_57A', 'avginstallast24m_3658937A', 'avglnamtstart24m_4525187A', 'avgoutstandbalancel6m_4187114A', 'avgpmtlast12m_4525200A', 'credamount_770A', 'currdebt_22A', 'currdebtcredtyperange_828A', 'disbursedcredamount_1113A', 'downpmt_116A', 'inittransactionamount_650A', 'lastapprcommoditycat_1041M', 'lastapprcommoditytypec_5251766M', 'lastapprcredamount_781A', 'lastcancelreason_561M', 'lastotherinc_902A', 'lastotherlnsexpense_631A', 'lastrejectcommoditycat_161M', 'lastrejectcommodtypec_5251769M', 'lastrejectcredamount_222A', 'lastrejectreason_759M', 'lastrejectreasonclient_4145040M', 'maininc_215A', 'maxannuity_159A', 'maxannuity_4075009A', 'maxdebt4_972A', 'maxinstallast24m_3658928A', 'maxlnamtstart6m_4525199A', 'maxoutstandbalancel12m_4187113A', 'maxpmtlast3m_4525190A', 'previouscontdistrict_112M', 'price_1097A', 'sumoutstandtotal_3546847A', 'sumoutstandtotalest_4493215A', 'totaldebt_9A', 'totalsettled_863A', 'totinstallas

In [10]:
base_train, X_train, y_train = from_polars_to_pandas(case_ids_train)
base_valid, X_valid, y_valid = from_polars_to_pandas(case_ids_valid)
base_test, X_test, y_test = from_polars_to_pandas(case_ids_test)

for df in [X_train, X_valid, X_test]:
    df = convert_strings(df)

In [11]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
category_colums = X_test.select_dtypes(include=["category"]).columns

# encode test data
encoded_data = encoder.fit_transform(X_test[category_colums])
encoded_df = pd.DataFrame(
    encoded_data.toarray(), columns=encoder.get_feature_names_out(category_colums)
)
X_test.drop(columns=category_colums, inplace=True)
X_test = pd.concat([X_test, encoded_df], axis=1)

# one hot encoding for X_train
encoded_data = encoder.fit_transform(X_train[category_colums])
encoded_df = pd.DataFrame(
    encoded_data.toarray(), columns=encoder.get_feature_names_out(category_colums)
)
X_train.drop(columns=category_colums, inplace=True)
X_train = pd.concat([X_train, encoded_df], axis=1)

# one hot encoding for X_valid
encoded_data = encoder.fit_transform(X_valid[category_colums])
encoded_df = pd.DataFrame(
    encoded_data.toarray(), columns=encoder.get_feature_names_out(category_colums)
)
X_valid.drop(columns=category_colums, inplace=True)
X_valid = pd.concat([X_valid, encoded_df], axis=1)

print(X_test.shape)
print(X_train.shape)
print(X_valid.shape)

(305332, 726)
(915995, 826)
(305332, 728)


In [12]:
# match columns with X_Test
test_columns = X_test.columns
missing_columns = [
    col
    for col in test_columns
    if col not in X_train.columns or col not in X_valid.columns
]
X_test = X_test.drop(columns=missing_columns)
test_columns = X_test.columns
X_train = X_train[test_columns]
X_valid = X_valid[test_columns]
print(X_test.shape)
print(X_train.shape)
print(X_valid.shape)

(305332, 657)
(915995, 657)
(305332, 657)


In [13]:
# missing data
for df in [X_train, X_valid, X_test]:
    missing_percentage = (df.isnull().sum() / len(df)) * 100
    columns_to_drop = missing_percentage[missing_percentage > 50].index.tolist()
    df.drop(columns=columns_to_drop, inplace=True)
print(f"Train: {X_train.shape}")
print(f"Valid: {X_valid.shape}")
print(f"Test: {X_test.shape}")

Train: (915995, 640)
Valid: (305332, 640)
Test: (305332, 640)


## Training XGBOOST

In [14]:
# Split the dataset into two equal parts
base_train1, base_train2, X_train1, X_train2, y_train1, y_train2 = train_test_split(
    base_train, X_train, y_train, test_size=0.2, random_state=42
)

base_train3, base_train4, X_train3, X_train4, y_train3, y_train4 = train_test_split(
    base_train2, X_train2, y_train2, test_size=0.2, random_state=42
)

In [36]:
base_train3.head(5)

,case_id,WEEK_NUM,target
803514,1933240,89,0
22029,36746,44,0
304738,892225,48,0
445744,1336630,14,0
538791,1492108,32,0
